# Construct a custom Environment for Pair Trading

Some examples on the market
* [custom env example](https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/5_custom_gym_env.ipynb#scrollTo=RqxatIwPOXe_)
* [StockTradingEnv by Adam King](https://github.com/notadamking/Stock-Trading-Environment)
* [FinRL](https://github.com/AI4Finance-Foundation/FinRL)

Target is to construct a custom Env for pair trading

This env restrict the behaviour of RL learner to pair trading only

In [1]:
import os
import pickle

from stable_baselines3 import PPO, A2C, DQN

from params import *
from utils.read2df import read2df, unify_dfs
from utils.clearlogs import clear_logs
from envs.env_rl_restrict_thres2 import RL_Restrict_TradeEnv

folder_path = f"result/rl-restrict-thres"
os.makedirs(folder_path, exist_ok=True)

Load data from `preliminaries.ipynb`

In [2]:
# with open('result/cointncorr.pickle', 'rb') as pk:
#     data = pickle.load(pk)

# with open('result/gridsearch/best_res.pickle', 'rb') as pk:
#     best_profit, best_params = pickle.load(pk)

# dfs = read2df(symbols=data[0], freqs={data[1]: freqs[data[1]]})

# tics, df = unify_dfs(dfs, symbols=data[0], period=best_params['period'])

# df.head(10)

In [3]:
# train = df[(df['datetime'] >= datetime.strptime(start_date, date_format)) & (df['datetime'] < datetime.strptime(trade_date, date_format))]
# test = df[(df['datetime'] >= datetime.strptime(trade_date, date_format)) & (df['datetime'] < datetime.strptime(end_date, date_format))]

# max_train_len = len(train)-best_params['period']-1
# print(f"The length of our training data: {len(train)}")

## Check with baselin3 `env_checker`

Check if the env meets the requirements of `stable_baseline3`

In [4]:
from stable_baselines3.common.env_checker import check_env
# > UserWarning: The action space is not based off a numpy array. Typically this means it's either a Dict or Tuple space. This type of action space is currently not supported by Stable Baselines 3. You should try to flatten the action using a wrapper.
# Baseline 3 does not support Dict/Tuple action spaces....only Box Discrete MultiDiscrete MultiBinary
# Is there another way to achieve the same functionality?

train = []

env = RL_Restrict_TradeEnv(train)
check_env(env)

In [5]:
log_path = f"logs/restrict_thres/"
clear_logs(log_path)

# Read more about tensorboard
# https://github.com/tensorflow/tensorboard/blob/master/README.md
# https://www.tensorflow.org/tensorboard/get_started

Deleted: logs/restrict_thres/PPO_1
Deleted: logs/restrict_thres/PPO_2
Deleted: logs/restrict_thres/PPO_3


In [7]:
'''PPO'''

env = RL_Restrict_TradeEnv(train)

model_ppo = PPO("MlpPolicy", env, verbose=1, gamma=1, tensorboard_log=log_path)
model_ppo.learn(total_timesteps=50000)#max_train_len)
model_ppo.save(f"{folder_path}/ppo_pairtrading")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to logs/restrict_thres/PPO_1
-----------------------------
| time/              |      |
|    fps             | 2216 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1312        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.020798888 |
|    clip_fraction        | 0.441       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.676      |
|    explained_variance   | -0.0101     |
|    learning_rate        | 0.0003      |
|    loss                 | 1.76        |
|    n_updates            | 10          |
|    policy_gradient_loss | -

In [10]:
try:
    os.remove(f"{folder_path}/networth_ppo.csv")
except OSError:
    pass

env = RL_Restrict_TradeEnv(train)
obs, _ = env.reset()

for i in range(20):
    env.render()
    action, _states = model_ppo.predict(obs)
    observation, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated
    if terminated:
        print("Test Finished!")
        break
    elif truncated:
        print("bankrupted!")
        break

AttributeError: 'RL_Restrict_TradeEnv' object has no attribute 'signal'